# Progetto di Machine Learning
#### Double inverted pendulum - Lorenzo Frangella 1899674

In [2]:
import gymnasium as gym
import math 
import os 
import torch
import torch.nn as nn
import pandas as pd